# Ullevål MRI protocol exploration

In [1]:
import pandas as pd
import pandera as pa
import numpy as np
import regex as re

* Load dataset and delete first empty column. 
* Name columns.
* Create column that indicates if child protocol or not

In [2]:
mr_protokoller = pd.read_csv('protokoller.csv')
mr_protokoller = mr_protokoller.iloc[:,1:]
mr_protokoller.columns = ['Protocol name', 'Nativ', 'Siemens', 'Philips', 'GE', 'Contrast-comments', 'CC-Siemens', 'CC-Philips', 'CC-GE', 'Post-contrast', 'PC-Siemens', 'PC-Philips', 'PC-GE',
       'AC-1', 'AC-2', 'AC-3', 'AC-4', 'AC-5', 'AV-6', 'AC-7', 'AC-8']
mr_protokoller.loc[mr_protokoller['Protocol name'].str.contains('BKC|Barn|BORB|BARN|BØNH', regex=True),'Barneprotokoll'] = 1
mr_protokoller.loc[mr_protokoller['Barneprotokoll'] != 1, 'Barneprotokoll'] = 0
mr_protokoller['Barneprotokoll'] = mr_protokoller['Barneprotokoll'].astype(int)
mr_protokoller = mr_protokoller.loc[mr_protokoller['Barneprotokoll'] == 0, :]


In [3]:
# Delete Loc from protocol
def del_loc(protocol_string):
    if protocol_string is np.nan:
        return protocol_string
    elif protocol_string[:3] == 'Loc':
        return protocol_string[4:]
    else:
        return protocol_string

### Start with Siemens adult protocols

In [4]:
mr_protokoller['Siemens'] = mr_protokoller['Siemens'].apply(del_loc) 
siemens_pre_contrast = mr_protokoller.iloc[:,:3]
siemens_pre_contrast

,Protocol name,Nativ,Siemens
0,MR KC1: Caput standard utredning (MRKC),Nativ,"Sag T1 3D mprage,Tra DWI,Sag 3D Flair FS,Tra T..."
1,MR KC2: Caput m/kontrast (MRKCK),Prekontrast,"Sag T1 3D Space,Tra DWI"
2,MR KC3: Caput Cerebralt infarkt(MRKCINF),Nativ,"Sag T1 TSE,Tra T2 4mm,Tra Flair 4mm,Tra SWI,T..."
3,MR KC4: Caput demyeliniserende sykdom (MRKCMS),Prekontrast,"Sag T1 3D Space,Tra DWI"
4,MR KC5: Caput Traume (MRKCTR),Nativ,"Sag T1 3D mprage,Sag 3D Flair,Tra T2 TSE,Tra S..."
...,...,...,...
57,MR PB2: Plexus brachialis traume,Nativ,"Cor T2 3D STIR Space,Cor T1 3D Space,Cor T2 3D..."
58,MR PB3: Plexus brachialis tumor,Prekontrast,"Cor T2,3D,STIR,Cor T1 3D Space,Cor T2 3D Space"
59,MR PL1: Plexus lumbosacralis standard,Nativ,"Cor T1 3D Vibe Dixon,Cor T2 3D Space STIR,Konf..."
60,MR PL2: Plexus lumbosacralis traume,Nativ,"Cor T1 3D Vibe Dixon,Cor T2 3D STIR Space"


In [5]:
# Make number of columns for dataframe that matches number of sequences
def make_columns(antall, word):
    n = 1
    columns = []
    for i in range(antall):
        name = word+'_'+str(n)
        n += 1
        columns.append(name)
    return columns 

In [6]:
siemens_antall_precontrast = siemens_pre_contrast['Siemens'].str.split(',', expand=True)
antall = len(siemens_antall_precontrast.columns)
siemens_antall_precontrast.columns = make_columns(antall, 'Siemens')
siemens_pre_contrast = siemens_pre_contrast[['Protocol name', 'Nativ']].join(siemens_antall_precontrast)
siemens_pre_contrast.to_csv('siemens.csv')
siemens_pre_contrast.head(4)

,Protocol name,Nativ,Siemens_1,Siemens_2,Siemens_3,Siemens_4,Siemens_5,Siemens_6,Siemens_7,Siemens_8,Siemens_9,Siemens_10,Siemens_11,Siemens_12,Siemens_13,Siemens_14
0,MR KC1: Caput standard utredning (MRKC),Nativ,Sag T1 3D mprage,Tra DWI,Sag 3D Flair FS,Tra T2 TSE,None,None,None,None,None,None,None,None,None,None
1,MR KC2: Caput m/kontrast (MRKCK),Prekontrast,Sag T1 3D Space,Tra DWI,None,None,None,None,None,None,None,None,None,None,None,None
2,MR KC3: Caput Cerebralt infarkt(MRKCINF),Nativ,Sag T1 TSE,Tra T2 4mm,Tra Flair 4mm,Tra SWI,Tra DWI resolve 4mm,None,None,None,None,None,None,None,None,None
3,MR KC4: Caput demyeliniserende sykdom (MRKCMS),Prekontrast,Sag T1 3D Space,Tra DWI,None,None,None,None,None,None,None,None,None,None,None,None


In [7]:
def split_sequence(elements):
    for key in data.keys():
        if isinstance(elements, str):
            #elements = sequence_string.split(' ')
            #for element in elements:
            if key == 'plane':
                search_plane(elements, key)
            elif key == 'sequence contrast':
                search_sequence_type(elements, key)
            else:
                continue
        else:
            data[key].append(np.nan)

In [26]:
def search_plane(element, key):
    result = re.search('sag|cor|tra|neu', element, re.IGNORECASE)
    if result:
        term = result.group(0)
        term = term.lower()
        term = term[0].upper()+term[1:]
        data[key].append(term)
    else:
        data[key].append(np.nan)

In [33]:
def search_sequence_type(element, key):
    result =  re.search('T1|T2|DWI|PD', element, re.IGNORECASE)
    if result:
        data[key].append(result.group(0))
    else:
        data[key].append(np.nan)
    

In [34]:
pandas_series = siemens_pre_contrast['Siemens_1']
data = {'plane':[], 'sequence contrast':[]}
for entry in pandas_series.tolist():
        split_sequence(entry)

In [35]:
test = 'Sag T1 3D mprage'
re.search('T1|T2', test).group(0)

'T1'

In [38]:
pd.DataFrame(data).to_csv('plane_contrast.csv')

In [40]:
results = pd.DataFrame(data)
results

,plane,sequence contrast
0,Sag,T1
1,Sag,T1
2,Sag,T1
3,Sag,T1
4,Sag,T1
...,...,...
57,Cor,T2
58,Cor,T2
59,Cor,T1
60,Cor,T1


In [13]:
len(sequence_contrast)

NameError: name 'sequence_contrast' is not defined

In [ ]:
test_string = 'Sag T1 3D mpgrage'
test_string.split(' ')

['Sag', 'T1', '3D', 'mpgrage']

In [ ]:
siemens_pre_contrast['Siemens_1'].value_counts()

Sag T1 3D mprage                 11
Sag T1 TSE                        9
                                  5
Sag T1 3D Space                   3
Cor T1 3D Vibe Dixon              3
Cor T2                            2
Cor T2 TSE Dixon 3mm (IP + W)     2
Tra / Sag loc                     2
Sag T2 TSE                        2
Tra DWI (b0 og b1000)             1
Sag T2 TSE Dixon                  1
Cor T2 3D STIR Space              1
Sagittal snittføring              1
Sag TSE PD DIXON 2 mm             1
Neutral stilling av c.col:        1
Sag T1 TSE  Dixon                 1
Sag T1 3D Vibe Dixon (FS)         1
Sag  T1 TSE                       1
Sag T1 SE                         1
Sag T1 TSE 4 mm                   1
Sag T1 Dixon                      1
Sag T1 3D Space Black Blood       1
Sag T1 3mm                        1
Sag T1 3D SPACE                   1
Tra 3D TOF                        1
Tra DWI 5 mm                      1
Cor 2D TOF                        1
Sag T2                      